In [1]:
# 금융상품 추천 Random Forest 모델링
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
df

,연령대분류,교육수준분류,사업농업소득,자본이득소득,연령,총소득,금융위험감수,저축여부,급여소득,금융위험회피,...,가구주성별,결혼상태,자녀수,직업분류1,총자산,LIQ,CDS,NMMF,STOCKS,RETQLIQ
0,5,3,0.0,0.0,70,38804.734469,0,3,0.0,0,...,2,2,2,3,957100.0,1,0,0,0,1
1,5,3,0.0,0.0,70,38264.278557,0,3,0.0,0,...,2,2,2,3,1067300.0,1,0,0,0,1
2,5,3,0.0,0.0,70,36102.454910,0,3,0.0,0,...,2,2,2,3,957200.0,1,0,0,0,1
3,5,3,0.0,0.0,70,33508.266533,0,3,0.0,0,...,2,2,2,3,828600.0,1,0,0,0,1
4,5,3,0.0,0.0,70,35561.998998,0,3,0.0,0,...,2,2,2,3,825600.0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22288,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22289,2,2,0.0,0.0,40,33508.266533,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22290,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22291,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [14]:
X = df.iloc[:,:-5]
y = df.iloc[:,-5:]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
print(f"훈련 데이터: {train_X.shape[0]}개")
print(f"테스트 데이터: {test_X.shape[0]}개")

훈련 데이터: 17834개
테스트 데이터: 4459개


In [29]:
rf_model = RandomForestClassifier(n_estimators=100,
                                    max_depth=10,          # 깊이 제한
                                    min_samples_split=50,  # 분할 최소 샘플
                                    min_samples_leaf=20,   # 리프 최소 샘플
                                    random_state=42)
rf_model.fit(train_X, train_y)

RandomForestClassifier(max_depth=10, min_samples_leaf=20, min_samples_split=50,
                       random_state=42)

In [30]:
train_pred = rf_model.predict(train_X) #과적합 여부 확인(훈련 성능)
test_pred = rf_model.predict(test_X) #일반화 성능

train_pred_proba = rf_model.predict_proba(train_X)
test_pred_proba = rf_model.predict_proba(train_X)

print(train_pred)

[[1 0 0 0 1]
 [1 0 1 1 1]
 [1 0 1 1 1]
 ...
 [1 0 0 0 1]
 [1 0 1 1 1]
 [1 0 0 1 1]]


In [31]:
print(train_pred_proba)

[array([[1.02034369e-04, 9.99897966e-01],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       ...,
       [9.89869573e-04, 9.99010130e-01],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00]]), array([[0.89115027, 0.10884973],
       [0.91000881, 0.08999119],
       [0.89234697, 0.10765303],
       ...,
       [0.83915265, 0.16084735],
       [0.88531607, 0.11468393],
       [0.91048582, 0.08951418]]), array([[0.90493644, 0.09506356],
       [0.29662645, 0.70337355],
       [0.39680355, 0.60319645],
       ...,
       [0.87647307, 0.12352693],
       [0.35339609, 0.64660391],
       [0.67147911, 0.32852089]]), array([[0.73238563, 0.26761437],
       [0.26181915, 0.73818085],
       [0.33546355, 0.66453645],
       ...,
       [0.807922  , 0.192078  ],
       [0.23187843, 0.76812157],
       [0.3716825 , 0.6283175 ]]), array([[0.22788962, 0.77211038],
       [0.02204336, 0.97795664],
       [0.17374048, 0.82625952],
    

In [32]:
print(test_pred)

[[1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 1 1]
 ...
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]]


In [33]:
print(test_pred_proba)

[array([[1.02034369e-04, 9.99897966e-01],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       ...,
       [9.89869573e-04, 9.99010130e-01],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00]]), array([[0.89115027, 0.10884973],
       [0.91000881, 0.08999119],
       [0.89234697, 0.10765303],
       ...,
       [0.83915265, 0.16084735],
       [0.88531607, 0.11468393],
       [0.91048582, 0.08951418]]), array([[0.90493644, 0.09506356],
       [0.29662645, 0.70337355],
       [0.39680355, 0.60319645],
       ...,
       [0.87647307, 0.12352693],
       [0.35339609, 0.64660391],
       [0.67147911, 0.32852089]]), array([[0.73238563, 0.26761437],
       [0.26181915, 0.73818085],
       [0.33546355, 0.66453645],
       ...,
       [0.807922  , 0.192078  ],
       [0.23187843, 0.76812157],
       [0.3716825 , 0.6283175 ]]), array([[0.22788962, 0.77211038],
       [0.02204336, 0.97795664],
       [0.17374048, 0.82625952],
    

In [34]:
target_names = {
    'LIQ': '유동성자산',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

print("\n📊 모델 성능 평가")
print("=" * 50)

# 각 타겟별 성능 평가
for i, target in enumerate(y.columns):
    print(f"\n ★{target_names.get(target, target)} ({target})")
    print("-" * 30)
    
    # 정확도
    train_acc = (train_pred[:, i] == train_y.iloc[:, i]).mean()
    test_acc = (test_pred[:, i] == test_y.iloc[:, i]).mean()
    
    print(f"훈련 정확도: {train_acc:.3f}")
    print(f"테스트 정확도: {test_acc:.3f}")

    # 각 타겟마다 별도로 출력 보장
    display(f"=== {target} 완료 ===")  # Jupyter용 강제 출력
    
    # AUC Score
    try:
        train_auc = roc_auc_score(train_y.iloc[:, i], train_pred_proba[i][:, 1])
        test_auc = roc_auc_score(test_y.iloc[:, i], test_pred_proba[i][:, 1])
        print(f"훈련 AUC: {train_auc:.3f}")
        print(f"테스트 AUC: {test_auc:.3f}")
    except:
        print("AUC 계산 불가 (클래스 불균형)")
    
    # Classification Report
    print(f"\n분류 보고서 (테스트 데이터):")
    print(classification_report(test_y.iloc[:, i], test_pred[:, i]))


📊 모델 성능 평가

 ★유동성자산 (LIQ)
------------------------------
훈련 정확도: 0.989
테스트 정확도: 0.989


'=== LIQ 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       1.00      0.25      0.40        68
           1       0.99      1.00      0.99      4391

    accuracy                           0.99      4459
   macro avg       0.99      0.62      0.70      4459
weighted avg       0.99      0.99      0.99      4459


 ★양도성예금증서 (CDS)
------------------------------
훈련 정확도: 0.923
테스트 정확도: 0.927


'=== CDS 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4135
           1       0.00      0.00      0.00       324

    accuracy                           0.93      4459
   macro avg       0.46      0.50      0.48      4459
weighted avg       0.86      0.93      0.89      4459


 ★비머니마켓펀드 (NMMF)
------------------------------
훈련 정확도: 0.888
테스트 정확도: 0.886


...

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      3613
           1       0.77      0.57      0.65       846

    accuracy                           0.89      4459
   macro avg       0.84      0.77      0.79      4459
weighted avg       0.88      0.89      0.88      4459


 ★주식보유 (STOCKS)
------------------------------
훈련 정확도: 0.831
테스트 정확도: 0.818


'=== STOCKS 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      3240
           1       0.70      0.60      0.64      1219

    accuracy                           0.82      4459
   macro avg       0.78      0.75      0.76      4459
weighted avg       0.81      0.82      0.81      4459


 ★퇴직준비금유동성 (RETQLIQ)
------------------------------
훈련 정확도: 0.867
테스트 정확도: 0.852


'=== RETQLIQ 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.86      0.76      0.81      1855
           1       0.84      0.91      0.88      2604

    accuracy                           0.85      4459
   macro avg       0.85      0.84      0.84      4459
weighted avg       0.85      0.85      0.85      4459



In [35]:
# 교차 검증으로 과적합 확인
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rf_model, X, y, cv=5)
print(f"CV 평균: {cv_scores.mean():.3f}")

CV 평균: 0.510


In [36]:
# SMOTE 적용 테스트
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y['STOCKS'])

cv_scores = cross_val_score(
    RandomForestClassifier(n_estimators=50, random_state=42),
    X_balanced, y_balanced, cv=5
)
print(f"SMOTE 적용 CV: {cv_scores.mean():.3f}")

SMOTE 적용 CV: 0.896


# SMOTE적용 후 RandomForest 모델링

In [37]:
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
df

,연령대분류,교육수준분류,사업농업소득,자본이득소득,연령,총소득,금융위험감수,저축여부,급여소득,금융위험회피,...,가구주성별,결혼상태,자녀수,직업분류1,총자산,LIQ,CDS,NMMF,STOCKS,RETQLIQ
0,5,3,0.0,0.0,70,38804.734469,0,3,0.0,0,...,2,2,2,3,957100.0,1,0,0,0,1
1,5,3,0.0,0.0,70,38264.278557,0,3,0.0,0,...,2,2,2,3,1067300.0,1,0,0,0,1
2,5,3,0.0,0.0,70,36102.454910,0,3,0.0,0,...,2,2,2,3,957200.0,1,0,0,0,1
3,5,3,0.0,0.0,70,33508.266533,0,3,0.0,0,...,2,2,2,3,828600.0,1,0,0,0,1
4,5,3,0.0,0.0,70,35561.998998,0,3,0.0,0,...,2,2,2,3,825600.0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22288,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22289,2,2,0.0,0.0,40,33508.266533,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22290,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22291,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0


In [38]:
X = df.iloc[:,:-5]
y = df.iloc[:,-5:]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
print(f"훈련 데이터: {train_X.shape[0]}개")
print(f"테스트 데이터: {test_X.shape[0]}개")

훈련 데이터: 17834개
테스트 데이터: 4459개


In [ ]:
# SMOTE 적용
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y['STOCKS'])

In [40]:
rf_model = RandomForestClassifier(n_estimators=100,
                                    max_depth=10,          # 깊이 제한
                                    min_samples_split=50,  # 분할 최소 샘플
                                    min_samples_leaf=20,   # 리프 최소 샘플
                                    random_state=42)
rf_model.fit(X_balanced, y_balanced)

RandomForestClassifier(max_depth=10, min_samples_leaf=20, min_samples_split=50,
                       random_state=42)

In [ ]:
train_pred = rf_model.predict(train_X) #과적합 여부 확인(훈련 성능)
test_pred = rf_model.predict(test_X) #일반화 성능

train_pred_proba = rf_model.predict_proba(train_X)
test_pred_proba = rf_model.predict_proba(train_X)
